In [1]:
from io import StringIO

import anndata
import numpy as np
import scanpy

outdir = "/home/olga/googledrive/TabulaMicrocebus/data/cross-species"


import pandas as pd

# Human from Google Drive

In [2]:
human_folder = '/home/olga/googledrive/TabulaMicrocebus/data/human-lung-cell-atlas--from-kyle'
! ls -lha $human_folder

total 30G
drwxrwxr-x 1 olga czb 4.0K Dec 13  2019 .
drwxrwxr-x 1 olga czb 4.0K Dec  2  2019 ..
-rw-rw-r-- 1 olga czb 6.5G Jan 12 19:40 droplet_normal_lung_blood_P1-3.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 13 16:48 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 14 16:55 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes__minimal_obs.h5ad
-rw-r--r-- 1 olga czb 6.5G Jan 13 12:55 droplet_normal_lung_blood_P1-3__with_barcode_counts.h5ad
-rw-rw-r-- 1 olga czb  12M Oct 28  2019 droplet_normal_lung_blood_P1-3_metadata.csv
-rw-rw-r-- 1 olga czb 3.3G Oct 30  2019 droplet_normal_lung_blood_P1-3_ncbi_GCT.csv
-rw-rw-r-- 1 olga czb 1.1G Oct 28  2019 facs_normal_lung_blood_P1-3_gencode_GCT.csv
-rw-rw-r-- 1 olga czb 5.7M Oct 28  2019 facs_normal_lung_blood_P1-3_metadata.csv


## Copy to a local directory so we're not IO bound

In [3]:
! df -h

Filesystem                           Size  Used Avail Use% Mounted on
udev                                1008G     0 1008G   0% /dev
tmpfs                                202G  3.1M  202G   1% /run
/dev/mapper/ndnd--vg-root            1.5T  582G  812G  42% /
tmpfs                               1008G  264K 1008G   1% /dev/shm
tmpfs                                5.0M     0  5.0M   0% /run/lock
tmpfs                               1008G     0 1008G   0% /sys/fs/cgroup
/dev/loop1                           141M  141M     0 100% /snap/gnome-3-26-1604/98
/dev/loop2                           2.3M  2.3M     0 100% /snap/gnome-system-monitor/145
/dev/loop3                           1.0M  1.0M     0 100% /snap/gnome-logs/100
/dev/loop5                           2.5M  2.5M     0 100% /snap/gnome-calculator/748
/dev/loop9                           141M  141M     0 100% /snap/gnome-3-26-1604/100
/dev/loop10                          1.0M  1.0M     0 100% /snap/gnome-logs/93
/dev/loop12               

In [4]:
human_folder2 = '/home/olga/TabulaMicrocebus/data/human-lung-cell-atlas--from-kyle'
! mkdir -p $human_folder2

In [5]:
# ! cp -r $human_folder $human_folder2

## Read in gene count table csvs

In [6]:
%time expression = pd.read_csv(f'{human_folder}/droplet_normal_lung_blood_P1-3_ncbi_GCT.csv')
print(expression.shape)
expression.head()

CPU times: user 49min 57s, sys: 3min 27s, total: 53min 25s
Wall time: 54min 2s
(26485, 65663)


,Unnamed: 0,P2_1_AAACCTGAGAAACCAT,P2_1_AAATGCCAGATGAGAG,P2_1_AACACGTTCGATCCCT,P2_1_AACACGTTCGCACTCT,P2_1_AACCATGCAGCTCGCA,P2_1_AACCGCGAGTCATCCA,P2_1_AACCGCGGTACTCTCC,P2_1_AACGTTGCATTTCACT,P2_1_AACTCAGTCCGTACAA,...,P3_6_GTTACAGCACGGCCAT,P3_6_TACGGGCGTTTGTGTG,P3_6_TCTGAGAAGAGTGAGA,P3_6_TGCGCAGTCCAAAGTC,P3_6_TGGGAAGCATGACGGA,P3_6_TGTCCCAGTTCTGAAC,P3_6_TTCTCCTTCTGTTTGT,P3_7_AACTCCCCACTACAGT,P3_7_TCGTAGATCATCGGAT,P3_4_GCTGCGAAGAGCTTCT
0,DDX11L1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,WASH7P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MIR6859-3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MIR6859-2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MIR6859-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
%time metadata = pd.read_csv(f'{human_folder}/droplet_normal_lung_blood_P1-3_metadata.csv')
print(metadata.shape)
metadata.head()

CPU times: user 154 ms, sys: 11.3 ms, total: 165 ms
Wall time: 3.63 s
(65662, 15)


,Unnamed: 0,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment
0,P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
1,P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
2,P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
3,P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
4,P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial


## Create Anndata object

In [8]:
expression.head()

,Unnamed: 0,P2_1_AAACCTGAGAAACCAT,P2_1_AAATGCCAGATGAGAG,P2_1_AACACGTTCGATCCCT,P2_1_AACACGTTCGCACTCT,P2_1_AACCATGCAGCTCGCA,P2_1_AACCGCGAGTCATCCA,P2_1_AACCGCGGTACTCTCC,P2_1_AACGTTGCATTTCACT,P2_1_AACTCAGTCCGTACAA,...,P3_6_GTTACAGCACGGCCAT,P3_6_TACGGGCGTTTGTGTG,P3_6_TCTGAGAAGAGTGAGA,P3_6_TGCGCAGTCCAAAGTC,P3_6_TGGGAAGCATGACGGA,P3_6_TGTCCCAGTTCTGAAC,P3_6_TTCTCCTTCTGTTTGT,P3_7_AACTCCCCACTACAGT,P3_7_TCGTAGATCATCGGAT,P3_4_GCTGCGAAGAGCTTCT
0,DDX11L1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,WASH7P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MIR6859-3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MIR6859-2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MIR6859-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
expression_index = expression.set_index('Unnamed: 0')
expression_index.index.name = 'gene_name'
print(expression_index.shape)
expression_index.head()

(26485, 65662)


,P2_1_AAACCTGAGAAACCAT,P2_1_AAATGCCAGATGAGAG,P2_1_AACACGTTCGATCCCT,P2_1_AACACGTTCGCACTCT,P2_1_AACCATGCAGCTCGCA,P2_1_AACCGCGAGTCATCCA,P2_1_AACCGCGGTACTCTCC,P2_1_AACGTTGCATTTCACT,P2_1_AACTCAGTCCGTACAA,P2_1_AACTCTTAGAACAACT,...,P3_6_GTTACAGCACGGCCAT,P3_6_TACGGGCGTTTGTGTG,P3_6_TCTGAGAAGAGTGAGA,P3_6_TGCGCAGTCCAAAGTC,P3_6_TGGGAAGCATGACGGA,P3_6_TGTCCCAGTTCTGAAC,P3_6_TTCTCCTTCTGTTTGT,P3_7_AACTCCCCACTACAGT,P3_7_TCGTAGATCATCGGAT,P3_4_GCTGCGAAGAGCTTCT
gene_name,,,,,,,,,,,,,,,,,,,,,
DDX11L1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WASH7P,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MIR6859-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MIR6859-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MIR6859-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
metadata.head()

,Unnamed: 0,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment
0,P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
1,P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
2,P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
3,P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
4,P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial


In [11]:
metadata_index = metadata.set_index('Unnamed: 0')
metadata_index.index.name = 'cell_id'
print(metadata_index.shape)
metadata_index.head()

(65662, 14)


,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment
cell_id,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial


In [12]:
human = anndata.AnnData(expression_index.T, obs=metadata_index)
human

AnnData object with n_obs × n_vars = 65662 × 26485 
    obs: 'nGene', 'nUMI', 'orig.ident', 'channel', 'tissue', 'region', 'percent.ribo', 'free_annotation', 'patient', 'sample', 'location', 'magnetic.selection', 'preparation.site', 'compartment'

## Clean "free annotation"

In [13]:
2+2

4

In [14]:
human.obs.free_annotation.value_counts().to_frame()

,free_annotation
Macrophage,14734
Capillary,7391
Natural Killer,4769
Capillary Aerocyte,4352
Alveolar Epithelial Type 2,3814
CD4+ Memory/Effector T,3013
Classical Monocyte,2077
CD8+ Naive T,2042
Pericyte,1608
Artery,1484


## Add cell ontology class column

In [15]:
s = '''free_annotation	n_cells	cell_ontology_class	permanent_url	ontology_lookup_service
Adventitial Fibroblast	557	adventitial cell	http://purl.obolibrary.org/obo/CL_0002503	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002503&viewMode=All&siblings=false
Airway Smooth Muscle	693	bronchial smooth muscle cell	http://purl.obolibrary.org/obo/CL_0002598	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002598
Alveolar Epithelial Type 1	972	type I pneumocyte	http://purl.obolibrary.org/obo/CL_0002062	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002062
Alveolar Epithelial Type 2	3814	type II pneumocyte	http://purl.obolibrary.org/obo/CL_0002063	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002063
Alveolar Fibroblast	1296	fibroblast of lung	http://purl.obolibrary.org/obo/CL_0002553	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002553&viewMode=All&siblings=false
Artery	1484	lung endothelial cell	http://purl.obolibrary.org/obo/CL_1001567	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1001567&viewMode=All&siblings=false
B	725	B cell	http://purl.obolibrary.org/obo/CL_0000236	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000236
Basal	436	basal cell	http://purl.obolibrary.org/obo/CL_0000646	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000646
Basophil/Mast 1	1350	myeloid leukocyte	http://purl.obolibrary.org/obo/CL_0000766	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000766&viewMode=All&siblings=false
Basophil/Mast 2	552	myeloid leukocyte	http://purl.obolibrary.org/obo/CL_0000766	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000766&viewMode=All&siblings=false
Bronchial Vessel 1	455	bronchial epithelial cell	http://purl.obolibrary.org/obo/CL_0002328	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002328
Bronchial Vessel 2	235	bronchial epithelial cell	http://purl.obolibrary.org/obo/CL_0002328	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002328
Capillary	7391	capillary endothelial cell	http://purl.obolibrary.org/obo/CL_0002144	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002144
Capillary Aerocyte	4352	capillary endothelial cell	http://purl.obolibrary.org/obo/CL_0002144	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002144
Capillary Intermediate 1	628	capillary endothelial cell	http://purl.obolibrary.org/obo/CL_0002144	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002144
Capillary Intermediate 2	464	capillary endothelial cell	http://purl.obolibrary.org/obo/CL_0002144	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002144
CD4+ Memory/Effector T	3013	CD4-positive, alpha-beta T cell	http://purl.obolibrary.org/obo/CL_0000624	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000624
CD4+ Naive T	932	naive thymus-derived CD4-positive, alpha-beta T cell	http://purl.obolibrary.org/obo/CL_0000895	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000895
CD8+ Memory/Effector T	1190	CD8-positive, alpha-beta T cell	http://purl.obolibrary.org/obo/CL_0000625	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000625
CD8+ Naive T	2042	naive thymus-derived CD8-positive, alpha-beta T cell	http://purl.obolibrary.org/obo/CL_0000900	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000900
Ciliated	1321	lung ciliated cell	http://purl.obolibrary.org/obo/CL_1000271	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000271
Classical Monocyte	2077	classical monocyte	http://purl.obolibrary.org/obo/CL_0000860	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000860
Club	931	club cell	http://purl.obolibrary.org/obo/CL_0000158	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000158
Differentiating Basal	265	basal cell	http://purl.obolibrary.org/obo/CL_0000646	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000646
EREG+ Dendritic	142	dendritic cell	http://purl.obolibrary.org/obo/CL_0000451	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000451
Fibromyocyte	98	muscle cell	http://purl.obolibrary.org/obo/CL_0000187	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000187
Goblet	159	lung goblet cell	http://purl.obolibrary.org/obo/CL_1000143	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000143
IGSF21+ Dendritic	280	dendritic cell	http://purl.obolibrary.org/obo/CL_0000451	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000451
Intermediate Monocyte	188	intermediate monocyte	http://purl.obolibrary.org/obo/CL_0002393	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002393
Ionocyte	22	ionocyte	http://purl.obolibrary.org/obo/CL_0005006	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0005006
Lipofibroblast	35	fibroblast	http://purl.obolibrary.org/obo/CL_0000057	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000057
Lymphatic	467	endothelial cell of lymphatic vessel	http://purl.obolibrary.org/obo/CL_0002138	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002138
Macrophage	14734	alveolar macrophage	http://purl.obolibrary.org/obo/CL_0000583	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000583
Mesothelial	29	mesothelial cell	http://purl.obolibrary.org/obo/CL_0000077	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000077
Mucous	491	mucus secreting cell	http://purl.obolibrary.org/obo/CL_0000319	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000319
Myeloid Dendritic Type 1	131	myeloid dendritic cell	http://purl.obolibrary.org/obo/CL_0000782	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000782
Myeloid Dendritic Type 2	266	myeloid dendritic cell	http://purl.obolibrary.org/obo/CL_0000782	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000782
Myofibroblast	248	myofibroblast cell	http://purl.obolibrary.org/obo/CL_0000186	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000186
Natural Killer	4769	natural killer cell	http://purl.obolibrary.org/obo/CL_0000623	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000623
Natural Killer T	335	mature NK T cell	http://purl.obolibrary.org/obo/CL_0000814	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000814
Neuroendocrine	11	lung neuroendocrine cell	http://purl.obolibrary.org/obo/CL_1000223	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000223&viewMode=All&siblings=false
Nonclassical Monocyte	804	non-classical monocyte	http://purl.obolibrary.org/obo/CL_0000875	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000875
OLR1+ Classical Monocyte	207	classical monocyte	http://purl.obolibrary.org/obo/CL_0000860	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000860
Pericyte	1608	pericyte cell	http://purl.obolibrary.org/obo/CL_0000669	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000669
Plasma	187	plasma cell	http://purl.obolibrary.org/obo/CL_0000786	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000786
Plasmacytoid Dendritic	137	plasmacytoid dendritic cell	http://purl.obolibrary.org/obo/CL_0000784	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000784
Platelet/Megakaryocyte	40	blood cell	http://purl.obolibrary.org/obo/CL_0000081	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000081&viewMode=All&siblings=false
Proliferating Basal	47	basal cell	http://purl.obolibrary.org/obo/CL_0000646	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000646
Proliferating Macrophage	226	alveolar macrophage	http://purl.obolibrary.org/obo/CL_0000583	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000583
Proliferating NK/T	105	immature NK T cell	http://purl.obolibrary.org/obo/CL_0000914	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000914
Proximal Basal	157	basal cell	http://purl.obolibrary.org/obo/CL_0000646	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000646
Proximal Ciliated	88	lung ciliated cell	http://purl.obolibrary.org/obo/CL_1000271	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_1000271
Serous	24	serous secreting cell	http://purl.obolibrary.org/obo/CL_0000313	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000313
Signaling Alveolar Epithelial Type 2	669	type II pneumocyte	http://purl.obolibrary.org/obo/CL_0002063	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002063
TREM2+ Dendritic	159	dendritic cell	http://purl.obolibrary.org/obo/CL_0000451	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000451
Vascular Smooth Muscle	469	vascular associated smooth muscle cell	http://purl.obolibrary.org/obo/CL_0000359	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0000359
Vein	1155	vein endothelial cell	http://purl.obolibrary.org/obo/CL_0002543	https://www.ebi.ac.uk/ols/ontologies/cl/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCL_0002543'''
df = pd.read_csv(StringIO(s), sep='\t', index_col=0)
df.head()

,n_cells,cell_ontology_class,permanent_url,ontology_lookup_service
free_annotation,,,,
Adventitial Fibroblast,557,adventitial cell,http://purl.obolibrary.org/obo/CL_0002503,https://www.ebi.ac.uk/ols/ontologies/cl/terms?...
Airway Smooth Muscle,693,bronchial smooth muscle cell,http://purl.obolibrary.org/obo/CL_0002598,https://www.ebi.ac.uk/ols/ontologies/cl/terms?...
Alveolar Epithelial Type 1,972,type I pneumocyte,http://purl.obolibrary.org/obo/CL_0002062,https://www.ebi.ac.uk/ols/ontologies/cl/terms?...
Alveolar Epithelial Type 2,3814,type II pneumocyte,http://purl.obolibrary.org/obo/CL_0002063,https://www.ebi.ac.uk/ols/ontologies/cl/terms?...
Alveolar Fibroblast,1296,fibroblast of lung,http://purl.obolibrary.org/obo/CL_0002553,https://www.ebi.ac.uk/ols/ontologies/cl/terms?...


In [16]:
human.obs['cell_ontology_class'] = human.obs.free_annotation.map(df.cell_ontology_class)
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class
cell_id,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell


In [17]:
human.obs.cell_ontology_class.value_counts()

alveolar macrophage                                     14960
capillary endothelial cell                              12835
natural killer cell                                      4769
type II pneumocyte                                       4483
CD4-positive, alpha-beta T cell                          3013
classical monocyte                                       2284
naive thymus-derived CD8-positive, alpha-beta T cell     2042
myeloid leukocyte                                        1902
pericyte cell                                            1608
lung endothelial cell                                    1484
lung ciliated cell                                       1409
fibroblast of lung                                       1296
CD8-positive, alpha-beta T cell                          1190
vein endothelial cell                                    1155
type I pneumocyte                                         972
naive thymus-derived CD4-positive, alpha-beta T cell      932
club cel

## Add "species" to obs

In [18]:
human.obs['species'] = "Human"
human.obs['species_latin'] = 'Homo sapiens'
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin
cell_id,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens


## Set Tissue to be capitalized

In [19]:
human.obs.tissue = human.obs.tissue.str.capitalize()
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin
cell_id,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens


## Write h5ad

In [20]:
ls -lha $human_folder

total 30G
drwxrwxr-x 1 olga czb 4.0K Dec 13  2019 ./
drwxrwxr-x 1 olga czb 4.0K Dec  2  2019 ../
-rw-rw-r-- 1 olga czb 6.5G Jan 12 19:40 droplet_normal_lung_blood_P1-3.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 13 16:48 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 14 16:55 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes__minimal_obs.h5ad
-rw-r--r-- 1 olga czb 6.5G Jan 13 12:55 droplet_normal_lung_blood_P1-3__with_barcode_counts.h5ad
-rw-rw-r-- 1 olga czb  12M Oct 28  2019 droplet_normal_lung_blood_P1-3_metadata.csv
-rw-rw-r-- 1 olga czb 3.3G Oct 30  2019 droplet_normal_lung_blood_P1-3_ncbi_GCT.csv
-rw-rw-r-- 1 olga czb 1.1G Oct 28  2019 facs_normal_lung_blood_P1-3_gencode_GCT.csv
-rw-rw-r-- 1 olga czb 5.7M Oct 28  2019 facs_normal_lung_blood_P1-3_metadata.csv


In [21]:
human.write(f"{human_folder}/droplet_normal_lung_blood_P1-3.h5ad")

... storing 'orig.ident' as categorical
... storing 'channel' as categorical
... storing 'tissue' as categorical
... storing 'region' as categorical
... storing 'free_annotation' as categorical
... storing 'sample' as categorical
... storing 'location' as categorical
... storing 'magnetic.selection' as categorical
... storing 'preparation.site' as categorical
... storing 'compartment' as categorical
... storing 'cell_ontology_class' as categorical
... storing 'species' as categorical
... storing 'species_latin' as categorical


In [22]:
ls -lha $human_folder

total 30G
drwxrwxr-x 1 olga czb 4.0K Dec 13  2019 ./
drwxrwxr-x 1 olga czb 4.0K Dec  2  2019 ../
-rw-rw-r-- 1 olga czb 6.5G Feb  5 14:51 droplet_normal_lung_blood_P1-3.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 13 16:48 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes.h5ad
-rw-r--r-- 1 olga czb 6.2G Jan 14 16:55 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes__minimal_obs.h5ad
-rw-r--r-- 1 olga czb 6.5G Jan 13 12:55 droplet_normal_lung_blood_P1-3__with_barcode_counts.h5ad
-rw-rw-r-- 1 olga czb  12M Oct 28  2019 droplet_normal_lung_blood_P1-3_metadata.csv
-rw-rw-r-- 1 olga czb 3.3G Oct 30  2019 droplet_normal_lung_blood_P1-3_ncbi_GCT.csv
-rw-rw-r-- 1 olga czb 1.1G Oct 28  2019 facs_normal_lung_blood_P1-3_gencode_GCT.csv
-rw-rw-r-- 1 olga czb 5.7M Oct 28  2019 facs_normal_lung_blood_P1-3_metadata.csv


### Get sequencing run assignments

In [23]:
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin
cell_id,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens


In [24]:
patient1_seq_run = '171205_A00111_0088_BH55NYDMXX'
everything_else_seq_run = '180607_A00111_0155_BHFCWYDMXX'

human.obs['sequencing_run'] = everything_else_seq_run
human.obs.loc[human.obs.channel.str.startswith('P1'), 'sequencing_run'] = patient1_seq_run
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin,sequencing_run
cell_id,,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX


In [25]:
human.obs.tissue.value_counts()

Lung     60993
Blood     4669
Name: tissue, dtype: int64

In [26]:
human.obs.tail()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin,sequencing_run
cell_id,,,,,,,,,,,,,,,,,,
P3_6_TGTCCCAGTTCTGAAC,953,1525,Human Lung 10x - Patient 3,P3_6,Lung,normal,0.091803,Pericyte,3,distal 3,distal,epithelial,biohub,stromal,pericyte cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P3_6_TTCTCCTTCTGTTTGT,999,1480,Human Lung 10x - Patient 3,P3_6,Lung,normal,0.065541,Pericyte,3,distal 3,distal,epithelial,biohub,stromal,pericyte cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P3_7_AACTCCCCACTACAGT,651,1154,Human Lung 10x - Patient 3,P3_7,Lung,normal,0.178510,Pericyte,3,distal 3,distal,immune and endothelial,biohub,stromal,pericyte cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P3_7_TCGTAGATCATCGGAT,1050,1861,Human Lung 10x - Patient 3,P3_7,Lung,normal,0.141322,Pericyte,3,distal 3,distal,immune and endothelial,biohub,stromal,pericyte cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX
P3_4_GCTGCGAAGAGCTTCT,2218,4877,Human Lung 10x - Patient 3,P3_4,Lung,normal,0.078532,Pericyte,3,proximal 3,proximal,stromal,biohub,stromal,pericyte cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX


In [27]:
human.obs['cell_barcode'] = human.obs.index.str.split('_').str[-1]
human.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin,sequencing_run,cell_barcode
cell_id,,,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AAACCTGAGAAACCAT
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AAATGCCAGATGAGAG
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACACGTTCGATCCCT
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACACGTTCGCACTCT
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACCATGCAGCTCGCA


In [28]:
human_seq_run_to_barcode = human.obs.groupby(['sequencing_run', 'cell_barcode']).size()
print(human_seq_run_to_barcode.shape)
human_seq_run_to_barcode.head()

(63702,)


sequencing_run                 cell_barcode    
171205_A00111_0088_BH55NYDMXX  AAACCTGAGAGTTGGC    1
                               AAACCTGAGCGATAGC    1
                               AAACCTGAGGCAAAGA    1
                               AAACCTGAGGCATTGG    1
                               AAACCTGAGGCCCTTG    1
dtype: int64

In [29]:
human_seq_run_to_barcode.value_counts()

1    61830
2     1808
3       40
4       24
dtype: int64

In [30]:
human_single_copy_barcodes = human_seq_run_to_barcode[human_seq_run_to_barcode == 1]
print(human_single_copy_barcodes.shape)
human_single_copy_barcodes.head()

(61830,)


sequencing_run                 cell_barcode    
171205_A00111_0088_BH55NYDMXX  AAACCTGAGAGTTGGC    1
                               AAACCTGAGCGATAGC    1
                               AAACCTGAGGCAAAGA    1
                               AAACCTGAGGCATTGG    1
                               AAACCTGAGGCCCTTG    1
dtype: int64

In [31]:
human_single_copy_barcodes.name = 'n_barcodes'

In [32]:
obs_with_barcode_counts = human.obs.join(human_single_copy_barcodes, on=['sequencing_run', 'cell_barcode'])
obs_with_barcode_counts.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,cell_ontology_class,species,species_latin,sequencing_run,cell_barcode,n_barcodes
cell_id,,,,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AAACCTGAGAAACCAT,1.0
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AAATGCCAGATGAGAG,1.0
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACACGTTCGATCCCT,1.0
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACACGTTCGCACTCT,1.0
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,Lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,capillary endothelial cell,Human,Homo sapiens,180607_A00111_0155_BHFCWYDMXX,AACCATGCAGCTCGCA,1.0


## Make human h5ad adata with barcode counts

In [33]:
human_with_barcode_counts = human.copy()
human_with_barcode_counts.obs = obs_with_barcode_counts

### Write the h5ad with barcode counts

In [34]:
h5ad = f"{human_folder}/droplet_normal_lung_blood_P1-3__with_barcode_counts.h5ad"
human_with_barcode_counts.write(h5ad)


... storing 'sequencing_run' as categorical
... storing 'cell_barcode' as categorical


In [35]:
human_no_duplicate_barcodes = human[obs_with_barcode_counts.n_barcodes == 1]
human_no_duplicate_barcodes

View of AnnData object with n_obs × n_vars = 61830 × 26485 
    obs: 'nGene', 'nUMI', 'orig.ident', 'channel', 'tissue', 'region', 'percent.ribo', 'free_annotation', 'patient', 'sample', 'location', 'magnetic.selection', 'preparation.site', 'compartment', 'cell_ontology_class', 'species', 'species_latin', 'sequencing_run', 'cell_barcode'

In [36]:
sorted(human_no_duplicate_barcodes.obs.columns)

['cell_barcode',
 'cell_ontology_class',
 'channel',
 'compartment',
 'free_annotation',
 'location',
 'magnetic.selection',
 'nGene',
 'nUMI',
 'orig.ident',
 'patient',
 'percent.ribo',
 'preparation.site',
 'region',
 'sample',
 'sequencing_run',
 'species',
 'species_latin',
 'tissue']

## Add "individual" column like `human_1_hlca1`

In [37]:
human_no_duplicate_barcodes.obs['individual'] = 'human_' + human_no_duplicate_barcodes.obs['patient'].astype(str) + '_hlca' +  human_no_duplicate_barcodes.obs['patient'].astype(str)
human_no_duplicate_barcodes.obs['individual'].value_counts()

Trying to set attribute `.obs` of view, making a copy.


human_2_hlca2    26894
human_3_hlca3    25382
human_1_hlca1     9554
Name: individual, dtype: int64

## Add age

Excerpt from human lung cell atlas paper
> Patient 1 was a 75 year-old male with a remote history of smoking, diagnosed with early stage adenocarcinoma who underwent left upper lobe (LUL) lobectomy; two blocks of normal tissue were obtained from lung periphery (“Distal 1a and 1b”). Patient 2 was a 46 year-old male, non-smoker with a right middle lobe (RML) endobronchial carcinoid, who underwent surgical resection of the right upper and middle lobes; two blocks of tissue were selected from mid-bronchial region (“Medial 2”) and periphery (“Distal 2”) of right upper lobe (RUL). Patient 3 was a 51 year-old female

In [38]:

human_age_mapper = {1: '75y', 2: '46y', 3: '51y'}
human_no_duplicate_barcodes.obs['age'] = human_no_duplicate_barcodes.obs['patient'].map(human_age_mapper)
human_no_duplicate_barcodes.obs['age'].value_counts()

46y    26894
51y    25382
75y     9554
Name: age, dtype: int64

## Add sex for mouse lemurs and humans

In [39]:

human_sex_mapper = {1: 'male', 2: 'male', 3: 'female'}
human_no_duplicate_barcodes.obs['sex'] = human_no_duplicate_barcodes.obs['patient'].map(human_sex_mapper)

human_no_duplicate_barcodes.obs.groupby([ 'sex']).size()

sex
female    25382
male      36448
dtype: int64

### Write human file without duplicate barcodes

In [40]:
%%time

h5ad = f"{human_folder}/droplet_normal_lung_blood_P1-3__no_duplicate_barcodes.h5ad"
human_no_duplicate_barcodes.write(h5ad)


... storing 'sequencing_run' as categorical
... storing 'cell_barcode' as categorical
... storing 'individual' as categorical
... storing 'age' as categorical
... storing 'sex' as categorical


CPU times: user 2.43 s, sys: 6.54 s, total: 8.96 s
Wall time: 3min 15s
